In [1]:
import pandas as pd
import numpy as np
from utils.personal_utils import *

In [2]:
# Load single ts
data_path = '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TRAIN.ts'
ts_1 = load_single_ts(data_path, 5, 3)
matrix_appliances = load_datapoint(data_path, 0)

119it [00:03, 32.61it/s]
100%|██████████| 95/95 [00:00<00:00, 563.19it/s]
119it [00:03, 31.99it/s]
100%|██████████| 95/95 [00:00<00:00, 574.25it/s]


In [3]:
# Quantization Testing: Test the returnd dtyps of the transforms

import pandas as pd
import numpy as np

from scipy.fft import dct, idct, fft, ifft
import pywt


print(ts_1.dtype)
ts_1 = ts_1.astype(np.float32)

print(dct(ts_1, norm='ortho',type=2).dtype)
print(fft(ts_1, norm='ortho').real.dtype)
print(pywt.dwt(ts_1, 'db4')[1].dtype)


float64
float32
float32
float32


In [5]:
data_path2 = '/home/sim/Desktop/TS Extrinsic Regression/data/AppliancesEnergy_TRAIN.ts'
matrix_ieee = load_datapoint(data_path2, 90)
ts_ie = load_single_ts(data_path2, 5, 3)

0it [00:00, ?it/s]

119it [00:03, 30.24it/s]
100%|██████████| 95/95 [00:00<00:00, 538.52it/s]
119it [00:03, 33.05it/s]
100%|██████████| 95/95 [00:00<00:00, 849.59it/s]


In [6]:
#Test general shape of compression
dr = 0.4
print(ts_ie[:100])
print(dwt_compress(ts_ie[0:100], dr ))
#print(compute_rmse(ts_ie, dwt_compress(ts_ie, dr )))

[39.96666667 40.1        40.26666667 40.52666667 40.89       41.16333333
 41.4        41.4        41.53       41.59       41.79       41.93
 42.06666667 42.33333333 42.65666667 42.86333333 43.09       43.2675
 43.46666667 43.73       43.93       44.09       44.16333333 44.36
 44.56       44.7        44.7        44.9        44.82666667 44.79
 44.79       44.76       44.7        44.79       44.86333333 44.9
 44.9        44.95       45.         45.06       45.2        45.26
 45.23       45.29       45.4        45.46666667 45.53       45.59
 45.66333333 45.66333333 45.73       45.79       45.9        45.96666667
 46.         46.         46.         46.12666667 46.2        46.2
 46.26       46.29       46.36333333 46.36333333 46.36333333 46.43333333
 46.5        46.59       46.66333333 46.79       46.73       46.7
 46.7        46.7        46.7        46.66333333 46.59       46.59
 46.59       46.53       46.46666667 46.4        46.53       46.59
 46.86       47.19333333 47.36       47.5    

NameError: name 'dwt_compress' is not defined

In [36]:
# Compute dft on every column of matrix
matrix = matrix_ieee

matrix_compressed = np.zeros(matrix.shape)
for i in range(matrix.shape[1]):
    matrix_compressed[:,i] = dwt_compress(matrix[:,i], 0.65)


compute_rmse_of_datapoint(matrix, matrix_compressed)

#(1.4435339091230222,
#array([3.71996501, 3.38425485, 0.03060634, 0.0288563 , 0.05398704]))

(0.9745457315927168,
 array([2.51989208, 2.29014343, 0.01868709, 0.01885416, 0.02515189]))

Testing calculateCompRatio

In [ ]:
# Testing calculateCompRatio
import gzip

def calculateCompRatio(dataset_array, array_flatdim_comp):

    # Calculate the number of bytes in the raw dataset array
    num_bytes_raw = dataset_array.nbytes
    print("Number of bytes in the raw dataset array:", num_bytes_raw)
    print("Number of the bytes in array_comp before gzip", array_flatdim_comp.nbytes)
    # Flatten the compressed array in column-major order. # Tested it, not need to put .copy() after.
    array_flatdim_1d = array_flatdim_comp.reshape(-1, order='F')
    print("Array flattened", array_flatdim_1d)

    # Convert the flattened array to a byte string
    compressed_data_bytestring = array_flatdim_1d.tobytes()

    print("Size of raw_data after gzip", len(gzip.compress(dataset_array.tobytes())))

    # Compress the byte string using gzip compression
    compressed_data_gzipd = gzip.compress(compressed_data_bytestring, compresslevel=9)
    print("Size of the compressed data after gzip:", len(compressed_data_gzipd))



    # Calculate and return the compression ratio
    compression_ratio = num_bytes_raw / len(compressed_data_gzipd)
    print("Compression ratio:", compression_ratio)
    return compression_ratio


In [ ]:
# Try with easy data. Then try with real time-series.

num_dp = 3
len_ts = 9
num_dim = 4

array = np.arange(num_dp * len_ts * num_dim)
array_comp = np.arange(num_dp * len_ts * num_dim)



array_comp = array_comp.reshape(num_dp * len_ts, num_dim)

array_comp[[1,2,3,5,6,7,9,12,13,14,15,16,15,16,17,20]] = 0


print(array_comp)
calculateCompRatio(array, array_comp)


# Conclusions:
# Values look good! Show that also with zeroing out we reached a little more compression!